In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F

In [2]:
def classification_metrics(target, pred):
    tn, fp, fn, tp = metrics.confusion_matrix(target, pred).ravel()
    acc = (tn + tp) / (tn + fp +fn + tp)
    sen = tp / (tp + fn)
    spc = tn / (tn + fp)
    prc = tp / (tp + fp)
    return acc, sen, spc, prc

In [3]:
data = load_breast_cancer()
X, y = data['data'], data['target']
X_norm = (X - X.mean(axis=0)) / X.std(axis=0)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

In [5]:
# Make input and output as a torch.tensor object
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(dim=1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(dim=1)

X_train.shape, y_train.shape

(torch.Size([455, 30]), torch.Size([455, 1]))

In [6]:
class MyMLP(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.linear1 = nn.Linear(in_features=n_input, out_features=n_hidden)
        self.linear2 = nn.Linear(in_features=n_hidden, out_features=n_output)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        y = self.linear1(x)
        y = self.relu(y)
        y = self.linear2(y)
        y = self.sigmoid(y)
        return y
    

In [7]:
n_input, n_hidden, n_output = 30, 5, 1
model = MyMLP(n_input, n_hidden, n_output)
# model.fit()
# y_pred = model.predict(X_test)

In [8]:
# Define loss
loss_function = nn.BCELoss()      # for binary classification
# loss_function = nn.MSELoss()    # for regression

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
epochs = 200

for i in range(epochs):
    model.train()                        # Put model in the training mode
    optimizer.zero_grad()                # make all graidents zero
    y_pred = model(X_train)               # predict output (forward pass)
    loss = loss_function(y_pred, y_train) # calculate loss
    loss.backward()                      # backpropagation (backward pass)
    optimizer.step()                     # update parameters

    if i % 20 == 0:
        print(f'Loss = {loss.item():.3f}')

Loss = 0.626
Loss = 0.179
Loss = 0.099
Loss = 0.074
Loss = 0.060
Loss = 0.053
Loss = 0.047
Loss = 0.043
Loss = 0.039
Loss = 0.035


In [10]:
model.eval()
y_pred = model(X_test)
y_pred = (y_pred > 0.5).float()

classification_metrics(y_test.numpy(), y_pred.detach().numpy())

(0.9824561403508771,
 0.9859154929577465,
 0.9767441860465116,
 0.9859154929577465)